In [1]:
from annotation_parser import parse_annotations
from data_visualization import show_stats

PATH_ACTIONS                        = "../Data/Unprocessed/actions.xml"
PATH_ACTIONS_FILTERED               = "../Data/tmp/actions_filtered.csv"

MINIMUM_DURATION                    = 4
MINIMUM_ACTION_COUNT                = 5

In [2]:
df = parse_annotations(PATH_ACTIONS)

df["duration"] = df["end_frame"] - df["start_frame"] + 1
df_actions = df[~df["action"].isin(["ATTACK_PREPARATION", "OTHER_NO_ACTION", "OTHER_RECOVERY"])]

show_stats(df_actions)

action,min,max,mean,std,count
ATTACK_LUNGE,2,11,7.151163,1.482983,172
ATTACK_STOP_CUT,2,15,5.067797,2.016008,59
ATTACK_STEP_CUT,3,8,5.333333,1.314257,45
ATTACK_BEAT,3,10,5.085714,1.597267,35
ATTACK_COUNTER,2,7,4.758621,1.184880,29
ATTACK_RIPOSTE,2,11,5.058824,2.135140,17
ATTACK_FEINT,3,6,4.300000,0.823273,10
ATTACK_FLUNGE,5,11,8.200000,2.201010,10
ATTACK_REMISE,4,10,6.666667,2.338090,6
ATTACK_COMPOUND,5,17,10.000000,6.244998,3


Total Offensive Frames: 389
Total Defensive Frames: 128

Total Frames: 517
Total classes:  19



In [3]:
df_actions_short = df_actions[df_actions["duration"] < MINIMUM_DURATION]

df_filtered = df_actions[~df_actions["duration"].isin(df_actions_short["duration"])]
df_filtered = df_filtered[df_filtered.groupby("action")["action"].transform("size") >= MINIMUM_ACTION_COUNT]

show_stats(df_filtered)

df_filtered["file"] = df_filtered["source_id"] + "/" + df_filtered["clip_id"]
df_filtered.sort_values(by=["file", "fencer", "start_frame"], inplace=True)

df_filtered = df_filtered.reset_index(drop=True)
df_filtered["action_id"] = df_filtered.index.astype(int)

df_filtered = df_filtered[["file", "fencer", "action_id", "action", "start_frame", "end_frame"]]
df_filtered.to_csv(PATH_ACTIONS_FILTERED, index=False)

action,min,max,mean,std,count
ATTACK_LUNGE,4,11,7.205882,1.401171,170
ATTACK_STOP_CUT,4,15,5.460000,1.940019,50
ATTACK_STEP_CUT,4,8,5.560976,1.141245,41
ATTACK_BEAT,4,10,5.281250,1.528954,32
ATTACK_COUNTER,4,7,5.000000,0.979796,26
ATTACK_RIPOSTE,4,11,5.769231,1.921538,13
ATTACK_FLUNGE,5,11,8.200000,2.201010,10
ATTACK_FEINT,4,6,4.444444,0.726483,9
ATTACK_REMISE,4,10,6.666667,2.338090,6
action,min,max,mean,std,count


Total Offensive Frames: 357
Total Defensive Frames: 118

Total Frames: 475
Total classes:  14

